# ADES processes

## OGC API Features

### Install dependencies

In [ ]:
!pip3 install requests
!pip3 install -U --force-reinstall charset-normalizer

### Import libraries

In [ ]:
import requests

### Service variables

In [ ]:
url = "https://iliad-registry.inesctec.pt/collections/apkg/items?f=json"
process_name = "bathymetry_pipeline"

### Search processes

In [ ]:
headers = {
    "Content-Type": "application/query-cql-json"
}

payload = None

r = requests.get(url+f"&name={process_name}", headers=headers, json=payload)
response = r.json()
for i in response["features"]:
    id = i["id"]
    p_type = i["properties"]["class"]
    name = i["properties"]["name"]
    description = i["properties"]["description"]
    version = i["properties"]["software_version"]
    code_url = i["properties"]["code_repository"]
    print(f"\t- {id}")
    print(f"\t     {p_type}")
    print(f"\t     {name}")
    print(f"\t     {description}")
    print(f"\t     @{version}")
    print(f"\t     {code_url}")

## OGC API Processes

### Install dependencies

In [ ]:
!pip3 install requests
!pip3 install -U --force-reinstall charset-normalizer

### Import libraries

In [ ]:
import requests
from time import sleep
import json

### Service variables

In [ ]:
url = "https://iliad-processes.terradue.com"
endpoint = f"{url}/inesctec/wps3"

### Search processes

In [ ]:
r = requests.get(f"{endpoint}/processes", headers={'accept':'application/json'})
assert r.status_code == 200 and r.reason == 'OK', f'ERROR: request failed: {r.status_code} {r.reason} \n{endpoint}/processes'

response = r.json()

print("# Processes found:")
for i in response["processes"]:
    name = i["id"]
    title = i["title"]
    description = i["description"]
    version = i["version"]
    print(f"\t- {name}")
    print(f"\t     {title}")
    print(f"\t     {description}")
    print(f"\t     @{version}")

### Deploy a process
Edit the next block to insert the CWL URL that represents the process

In [ ]:
cwl_url = "https://raw.githubusercontent.com/ILIAD-ocean-twin/application_package/refs/heads/main/inesctec_apkgs/workflows/bathymetry-pipeline/bathymetry_pipeline_0_1_0.cwl"

In [ ]:
r = requests.post(f"{endpoint}/processes", json={"executionUnit": {'href': cwl_url, "type": "application/cwl"}})
assert r.status_code == 201 and r.reason == 'Created', f'ERROR: request failed: {r.status_code} {r.reason}'

response = r.json()

print("Process created!\n")
print("process_id: ", response['id'])
print("title: ", response['title'])
print("description: ", response['description'])
print("version: ", response['version'])

### Choose process and inputs
Edit the next block to select the process and inputs to create a job

In [ ]:
process_id = "bathymetry_pipeline"
f = open("./process_inputs.json", "r")
process_inputs = json.load(f)

### Execute job by process id

In [ ]:
r = requests.post(f"{endpoint}/processes/{process_id}", json={"inputs": process_inputs}, headers={"Prefer": "respond-async"})
assert r.status_code == 201 and r.reason == 'Created', f'ERROR: request failed: {r.status_code} {r.reason}'

location = r.headers['Location']
job_id = location.split("/")[4]

print(f"Process {process_id} executed")
print(f"\t{job_id}")

### Check job 
Edit the next block if you want to search for a different job_id, otherwise the last job_id will be used.

In [ ]:
# job_id = ""

In [ ]:
status = 'running'
wait_time = 20
while status == 'running':
  sleep(wait_time)
  r = requests.get(f"{endpoint}/jobs/{job_id}", headers={'accept':'application/json'})
  assert r.status_code == 200 and r.reason == 'OK', f'ERROR: request failed: {r.status_code} {r.reason}'

  response = r.json()

  status = response['status']
  print("job_id: ", response['jobID'])
  print("status: ", status)
  print("message: ", response['message'])


### Get job results
Edit the next block if you want to get the result of a different job_id, otherwise the last job_id will be used.

In [ ]:
# job_id = ""

In [ ]:
r = requests.get(f"{endpoint}/jobs/{job_id}/results", headers={'accept':'application/json'})

assert r.status_code == 200 and r.reason == 'OK', f'ERROR: request failed: {r.status_code} {r.reason}'

response = r.json()
base_path=response['StacCatalogUri'].split('/')[3]
print(base_path)

### Undeploy a process
Edit the next block to select the process that you want to undeploy

In [ ]:
# process_id = "bathymetry_pipeline"

In [ ]:
r = requests.delete(f"{endpoint}/processes/{process_id}", headers={'accept':'application/json'})
assert r.status_code == 204 and r.reason == 'No Content', f'ERROR: request failed: {r.status_code} {r.reason}'

## S3 storage - ADES

### Install dependencies

In [ ]:
!pip3 install boto3
!pip3 install pyyaml
!pip3 install pystac

### Connect s3 storage

In [ ]:
import boto3
import yaml
import os

config = yaml.safe_load(open('config.yaml', 'r'))
s3_region = config['s3']['region']
s3_endpoint = config['s3']['endpoint']
s3_access_key = config['s3']['access_key']
s3_secret_key = config['s3']['secret_key']
s3_bucket = config['s3']['bucket']
session  = boto3.Session(region_name=s3_region)
resource = session.resource('s3',
                endpoint_url=s3_endpoint,
                aws_access_key_id=s3_access_key,
                aws_secret_access_key=s3_secret_key)

bucket = resource.Bucket(s3_bucket)

### Print all objects

In [ ]:
for file in bucket.objects.all():
  print(file.key)

### Filter objects
Edit the next block to select a prefix to filter the objects

In [ ]:
#base_path = ''

In [ ]:
objs = bucket.objects.filter(Prefix=base_path)

for i in objs:
  print(i.key)

### Download Folder
Edit the next block to select a prefix to filter the objects

In [ ]:
# prefix = ''

In [ ]:
for obj in bucket.objects.filter(Prefix = base_path):
    if not os.path.exists(os.path.dirname(obj.key)):
        os.makedirs(os.path.dirname(obj.key))
    bucket.download_file(obj.key, obj.key)


# Download a file
# bucket.download_file('origin', 'destination')

### S3 metadata

In [ ]:
resource.meta.client.list_objects(Bucket=s3_bucket, Prefix='', Delimiter='/')